# CKq case study

This case study concerns the fluorescence measured from the core of the phycobilisome antenna in the quenched state (abbreviated CKq) of a cyanobacterium ([Tian et al. 2012][tian2012]).

The measurement and the global analysis are described in ([Tian et al. 2012][tian2012]).

The target analysis is based upon the model explained in Fig.12A of ([van Stokkum et al. 2018][vanstokkum2018]).

Below are Questions 1A-E, each worth 10 points.

<!-- References to papers -->

[tian2012]: https://doi.org/10.1016/j.bpj.2012.03.008 "Tian L, Gwizdala M, van Stokkum IHM, Koehorst RBM, Kirilovsky D, van Amerongen H (2012) Picosecond Kinetics of Light Harvesting and Photoprotective Quenching in Wild-Type and Mutant Phycobilisomes Isolated from the Cyanobacterium Synechocystis PCC 6803. Biophysical Journal 102:1692-1700"
[vanstokkum2018]: https://www.few.vu.nl/~ivo/pub/2018/2018PBSmodelPRES135_87.pdf "van Stokkum IHM, Gwizdala M, Tian L, Snellenburg JJ, van Grondelle R, van Amerongen H, Berera R (2018) A functional compartmental model of the Synechocystis PCC 6803 phycobilisome. Photosynthesis Research 135:87-102."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

CKq_data_path = "CKtr4q.ascii"
plot_data_overview(CKq_data_path, nr_of_data_svd_vectors=3);

## Global Analysis

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

CKq_global_model_path = "models/CKq-global-model.yml"
CKq_global_parameters_path = "models/CKq-global-parameters.yml"

#### Model file

In [ ]:
display_file(CKq_global_model_path, syntax="yaml")

#### Parameters file

The free parameters are the center (the location of the maximum of the IRF) and the width of the IRF, and the kinetic parameters.
Note that the dispersion parameters have been fixed (to save time), but they could have been estimated from the data as well.

Question 1A: estimate the missing starting values for the parameters from the data inspection, and insert these in the models/CKq-global-parameters.yml file. Estimate the parameters, and make sure you achieve a satisfactory fit.

In [ ]:
display_file(CKq_global_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.project.scheme import Scheme

CKq_global_scheme = Scheme(
    model=CKq_global_model_path,
    parameters=CKq_global_parameters_path,
    data={"CKq_data": CKq_data_path},
)
CKq_global_scheme.validate()

In [ ]:
from glotaran.optimization.optimize import optimize

CKq_global_result = optimize(CKq_global_scheme)

In [ ]:
CKq_global_result.root_mean_square_error

In [ ]:
CKq_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(CKq_global_result.data["CKq_data"])
plot_fitted_traces(CKq_global_result, wavelengths);

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    CKq_global_result,
    linlog=False,
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
);

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(CKq_global_result)

Question 1B: Comment on the interpretation of the EAS and the DAS (in relation to the estimated lifetimes), with the help of the course materials. Hint: to insert an asterisk to indicate an excited state, it must be preceded by a backslash, e.g. APC660\*

Answer 1B: 

## Dispersion curves

In [ ]:
CKq_global_result.data["CKq_data"].data.sel(time=slice(290, 350)).plot()
CKq_global_result.data["CKq_data"].irf_center_location.plot.line(x="spectral", add_legend=False)

## Target Analysis

### Used model and parameters

In [ ]:
CKq_target_model_path = "models/CKq-target-model.yml"
CKq_target_parameters_path = "models/CKq-target-parameters.yml"

#### Model file

In [ ]:
display_file(CKq_target_model_path, syntax="yaml")

#### Parameters file

The free parameters describe the IRF, the kinetics and the relative amount of the two megacomplexes.
Note that the dispersion parameters have been fixed, they could have been estimated from the data as well.
Compartments 1, 2 and 3 contain the same type of pigments, and thus have identical SAS.
Note that the first scaling parameter (0.75) is equal to the ratio of the number of pigments in the compartments 3 (18 pigments, cf. input3) and compartments 1 or 2 (24 pigments, cf. input s1_s2).
The second scaling parameter (0.3) has been fixed so that the areas of the SAS are similar. It could have been estimated using an equal area constraint.

Question 1C: Study the kinetic scheme CKq_scheme.png (in the CKq exam 2022 question 1 folder) and the model file carefully.
Insert the missing starting values in the models/CKq-target-parameters.yml file. Hint: use the above estimated kinetic and IRF parameters. Study the parameters file carefully.

In [ ]:
display_file(CKq_target_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
CKq_target_scheme = Scheme(
    model=CKq_target_model_path,
    parameters=CKq_target_parameters_path,
    data={"CKq_data": CKq_data_path},
)
CKq_target_scheme.validate()

In [ ]:
CKq_target_result = optimize(CKq_target_scheme)

In [ ]:
CKq_target_result.root_mean_square_error

In [ ]:
CKq_target_result.optimized_parameters

In [ ]:
compartments = CKq_target_scheme.model.initial_concentration["input1"].compartments

CKq_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
CKq_target_scheme.model.k_matrix["km2"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(CKq_target_result)

### Result plots

#### Fit quality

In [ ]:
plot_fitted_traces(CKq_target_result, wavelengths);

#### Overview

In [ ]:
plot_overview(
    CKq_target_result,
    linlog=False,
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
);

Question 1D: Compare the rmse of the global and of the target analysis. 
How can the rmse difference be explained. Hint: consider the amount of free parameters.

Comment on the interpretation of the SAS. Hint: compare to Fig.11B of ([van Stokkum et al. 2018][vanstokkum2018]).

Answer 1D: 




[vanstokkum2018]: https://www.few.vu.nl/~ivo/pub/2018/2018PBSmodelPRES135_87.pdf "van Stokkum IHM, Gwizdala M, Tian L, Snellenburg JJ, van Grondelle R, van Amerongen H, Berera R (2018) A functional compartmental model of the Synechocystis PCC 6803 phycobilisome. Photosynthesis Research 135:87-102."

Question 1E: Consider compartments 3 and 4. What is the enthalpy difference between these compartments? What is the entropy difference between these compartments? What is the Gibbs free energy difference between these compartments computed from the enthalpy and entropy? What is the Gibbs free energy difference between these compartments computed from the kinetic scheme? Show that these two computations of the Gibbs free energy difference are consistent with a margin of about 0.16 kB*T. Express all in units of kB*T (kB*T=25 meV=208 cm-1).

Answer 1E: 
